In [13]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor


train = pd.read_csv("../data/processed/train_postprocessed.csv")
test = pd.read_csv("../data/processed/test_postprocessed.csv")

In [3]:
y = np.log1p(train["SalePrice"])
X = train.drop(columns=["SalePrice", "Id"])

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_test = test.drop(columns=["Id"])

In [4]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(exclude=["int64", "float64"]).columns

In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

### Funkcja `rmse_cv` – walidacja krzyżowa

W celu rzetelnej oceny jakości modeli regresyjnych zastosowano
5-krotną walidację krzyżową (cross-validation).

Funkcja `rmse_cv` oblicza średni błąd RMSE (Root Mean Squared Error)
uzyskany w kolejnych podziałach danych.


In [6]:
def rmse_cv(model, X, y):
    scores = -cross_val_score(
        model,
        X,
        y,
        scoring="neg_root_mean_squared_error",
        cv=5
    )
    return scores.mean()

## Model 1: Regresja liniowa (baseline)

Regresja liniowa została użyta jako model bazowy (baseline),
stanowiący punkt odniesienia dla bardziej zaawansowanych metod.

Model ten zakłada:
- liniową zależność pomiędzy cechami a zmienną objaśnianą,
- brak silnej współliniowości pomiędzy predyktorami.

Ze względu na dużą liczbę cech i ich współzależności,
oczekuje się, że jego jakość predykcji będzie ograniczona.

In [7]:
lin_model = Pipeline([
    ("prep", preprocessor),
    ("model", LinearRegression())
])

rmse_lin = rmse_cv(lin_model, X, y)
rmse_lin

np.float64(0.1183764058176674)

## Model 2: Ridge Regression

Ridge Regression jest rozszerzeniem regresji liniowej
z regularyzacją typu L2.

Zastosowanie kary L2:
- zmniejsza wariancję modelu,
- ogranicza wpływ współliniowości,
- poprawia stabilność predykcji.

Model ten jest szczególnie odpowiedni dla danych
o dużej liczbie cech, takich jak zbiór Ames Housing.


In [8]:
ridge_model = Pipeline([
    ("prep", preprocessor),
    ("model", Ridge(alpha=10))
])

rmse_ridge = rmse_cv(ridge_model, X, y)
rmse_ridge

np.float64(0.10694987120079313)

## Model 3: ElasticNet

ElasticNet łączy regularyzację:
- L1 (Lasso) – selekcja cech,
- L2 (Ridge) – stabilizacja wag.

Pozwala to:
- automatycznie eliminować mniej istotne zmienne,
- zachować odporność na współliniowość.

Model ten stanowi kompromis pomiędzy Ridge i Lasso.


In [9]:
enet_model = Pipeline([
    ("prep", preprocessor),
    ("model", ElasticNet(alpha=0.001, l1_ratio=0.5))
])

rmse_enet = rmse_cv(enet_model, X, y)
rmse_enet

np.float64(0.10339412260221055)

## Model 4: Random Forest

Random Forest jest zespołem drzew decyzyjnych,
uczących się na losowych podpróbkach danych i cech.

Zalety:
- zdolność modelowania nieliniowości,
- odporność na obserwacje odstające,
- brak potrzeby skalowania danych.

Model ten często osiąga lepsze wyniki niż modele liniowe,
kosztem mniejszej interpretowalności.


In [10]:
rf_model = Pipeline([
    ("prep", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=300,
        random_state=42
    ))
])

rmse_rf = rmse_cv(rf_model, X, y)
rmse_rf

np.float64(0.12574799699617292)

## Model 5: Gradient Boosting

Gradient Boosting to metoda zespołowa,
w której kolejne modele uczą się na błędach poprzednich.

Charakterystyka:
- wysoka skuteczność predykcyjna,
- dobra kontrola nad overfittingiem,
- zdolność uchwycenia złożonych zależności.

Jest to jeden z najskuteczniejszych modeli
dla danych tabelarycznych.


In [11]:
gb_model = Pipeline([
    ("prep", preprocessor),
    ("model", GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.05,
        random_state=42
    ))
])

rmse_gb = rmse_cv(gb_model, X, y)
rmse_gb

np.float64(0.11233469577987858)

## Model 6: XGBoost

XGBoost jest zaawansowaną implementacją gradient boosting,
rozszerzoną m.in. o:
- regularyzację,
- losowe próbkowanie obserwacji i cech,
- optymalizację wydajności obliczeniowej.



In [14]:
xgb_model = Pipeline([
    ("prep", preprocessor),
    ("model", XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
])

rmse_xgb = rmse_cv(xgb_model, X, y)
rmse_xgb

np.float64(0.11197738824304686)

In [15]:
results = pd.DataFrame({
    "Model": [
        "Linear Regression",
        "Ridge",
        "ElasticNet",
        "Random Forest",
        "Gradient Boosting"
    ],
    "RMSE": [
        rmse_lin,
        rmse_ridge,
        rmse_enet,
        rmse_rf,
        rmse_gb
    ]
})

results.sort_values("RMSE")

,Model,RMSE
2,ElasticNet,0.103394
1,Ridge,0.106950
4,Gradient Boosting,0.112335
0,Linear Regression,0.118376
3,Random Forest,0.125748


## Model 7: Ensemble (model hybrydowy)

Ostatnim modelem jest model hybrydowy (ensemble),
będący średnią predykcji kilku najlepszych modeli bazowych.

Takie podejście pozwala:
- zmniejszyć wariancję predykcji,
- wykorzystać zalety różnych algorytmów,
- uzyskać bardziej stabilne wyniki.

Ensemble często przewyższa pojedyncze modele składowe.


In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

lin_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)
enet_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

pred_lr = lin_model.predict(X_val)
pred_ridge = ridge_model.predict(X_val)
pred_enet = enet_model.predict(X_val)
pred_rf = rf_model.predict(X_val)
pred_gb = gb_model.predict(X_val)
pred_xgb = xgb_model.predict(X_val)

# Ensemble
ensemble_pred = ( pred_enet + pred_gb) / 2

# Lista modeli i ich predykcji
models = ['Linear Regression', 'Ridge', 'ElasticNet', 'Random Forest', 
          'Gradient Boosting', 'XGBoost', 'Ensemble']
predictions = [pred_lr, pred_ridge, pred_enet, pred_rf, pred_gb, pred_xgb, ensemble_pred]

# Obliczanie metryk
results = []
for name, pred in zip(models, predictions):
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    mae = mean_absolute_error(y_val, pred)
    r2 = r2_score(y_val, pred)
    results.append([name, rmse, mae, r2])

# Tworzenie DataFrame i sortowanie po R2
df_results = pd.DataFrame(results, columns=['Model', 'RMSE', 'MAE', 'R2'])
df_results = df_results.sort_values(by='R2', ascending=False).reset_index(drop=True)
df_results


,Model,RMSE,MAE,R2
0,ElasticNet,0.097043,0.069834,0.892381
1,Ridge,0.099201,0.070536,0.887541
2,Ensemble,0.100410,0.069340,0.884782
3,Linear Regression,0.105716,0.074896,0.872286
4,Gradient Boosting,0.111384,0.078077,0.858222
5,Random Forest,0.111824,0.079441,0.857100
6,XGBoost,0.117059,0.077131,0.843406


In [17]:
#Zapis predykcji wszystkich modeli i wartości rzeczywistych

output_folder = "../data/predictions"
os.makedirs(output_folder, exist_ok=True)

metrics_path = os.path.join(output_folder, "model_metrics.csv")
df_results.to_csv(metrics_path, index=False)

preds_path = os.path.join(output_folder, "model_predictions.csv")

df_preds = pd.DataFrame({
    'y_true': y_val,
    'LinearRegression': pred_lr,
    'Ridge': pred_ridge,
    'ElasticNet': pred_enet,
    'RandomForest': pred_rf,
    'GradientBoosting': pred_gb,
    'XGBoost': pred_xgb,
    'Ensemble': ensemble_pred
})
df_preds.to_csv(preds_path, index=False)

### Ważność cech dla modeli drzewiastych

In [ ]:


# Folder na wyniki
output_folder = "../data/predictions"
os.makedirs(output_folder, exist_ok=True)

# Funkcja do pobierania nazw cech po preprocessing
def get_feature_names(column_transformer):
    feature_names = []
    for name, transformer, columns in column_transformer.transformers_:
        if transformer == 'drop':
            continue
        if hasattr(transformer, 'get_feature_names_out'):
            feature_names.extend(transformer.get_feature_names_out(columns))
        else:
            feature_names.extend(columns)
    return feature_names

# Modele drzewiaste w pipeline
tree_models = {
    "RandomForest": rf_model,
    "GradientBoosting": gb_model,
    "XGBoost": xgb_model
}

for model_name, pipeline in tree_models.items():

    preprocessor = pipeline.named_steps['prep']
    model = pipeline.named_steps['model']

    # Sprawdzenie czy model ma feature_importances_
    if not hasattr(model, 'feature_importances_'):
        continue

    # Nazwy cech
    feature_names = get_feature_names(preprocessor)

    # DataFrame z ważnością cech
    fi_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)

    # Zapis do CSV
    fi_path = os.path.join(output_folder, f"{model_name}_feature_importance.csv")
    fi_df.to_csv(fi_path, index=False)